In [1]:
# import all we need to compute the mAP and produce some plots
# implementing mAP as documented in The PASCAL Visual Object Classes (VOC) Challenge

import numpy as np
import pickle
import os
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# grab all computed results 
# (.pkl format from "darknet_evaluation_post_inference.py")

input_path = "C:/Users/Legos/Documents/PhD/FARTS/BENCHMARK/RESULTS_TERMITES"
input_files = []

for file in os.listdir(input_path):
    if file.endswith(".pkl"):
        input_files.append(os.path.join(input_path,file))
        
input_files.sort()
print("Found {} evaluation files.".format(len(input_files)))
for d, dataset in enumerate(input_files):
    print(d, dataset)
    
use_state = 0

Found 9 evaluation files.
0 C:/Users/Legos/Documents/PhD/FARTS/BENCHMARK/RESULTS_TERMITES\tr1_RESULTS.pkl
1 C:/Users/Legos/Documents/PhD/FARTS/BENCHMARK/RESULTS_TERMITES\tr2_RESULTS.pkl
2 C:/Users/Legos/Documents/PhD/FARTS/BENCHMARK/RESULTS_TERMITES\tr3_RESULTS.pkl
3 C:/Users/Legos/Documents/PhD/FARTS/BENCHMARK/RESULTS_TERMITES\tr4_RESULTS.pkl
4 C:/Users/Legos/Documents/PhD/FARTS/BENCHMARK/RESULTS_TERMITES\tr5_RESULTS.pkl
5 C:/Users/Legos/Documents/PhD/FARTS/BENCHMARK/RESULTS_TERMITES\ts1_RESULTS.pkl
6 C:/Users/Legos/Documents/PhD/FARTS/BENCHMARK/RESULTS_TERMITES\ts2_RESULTS.pkl
7 C:/Users/Legos/Documents/PhD/FARTS/BENCHMARK/RESULTS_TERMITES\ts3_RESULTS.pkl
8 C:/Users/Legos/Documents/PhD/FARTS/BENCHMARK/RESULTS_TERMITES\ts4_RESULTS.pkl


In [3]:
with open(input_files[use_state], 'rb') as f:
    data = pickle.load(f)

"""
- data[0][0]
- training_data_&_training_state

- - data[0][1][0]
- - threshold (for first dataset)
  
- - - data[0][1][1][0 1  2    3   4   5                  6] 
- - - dataset_name,   GT, TP, FN, FP, Average Precision, Recall
"""
#examples:

all_training_states = []

for elem in data:
    all_training_states.append(int(elem[0].split(".")[0].split("_")[-1]))

all_training_states.sort()
print(all_training_states)

[20000]


The goal is to retrieve the mean Average Precision (mAP) over 13 confidence thresholds ranging from 0.2 to 0.8, classifying a correct detection centre as being within 10% (of the image width) euclidean distance to a ground truth detection, disregarding multiple detections of the same object as they would be suppressed by non-maxmimum suppresion at run-time. We use this adjusted metric from the original, as the actual intersection over union is secondary to the agreement of centres, as different methods have been used to assign bounding boxes. Synthetically generated bounding boxes are defined as the smallest retangle including all projected 2D keypoints in the rendered images, whereas hand annotated bounding boxes are fixed, square detections, as a custom written centre tracking tool (BlenderMotionExport) was used to semi-automatically produce these datasets.

As an example we will plot the precision over recall for these 13 thresholds for the first snapshot of the imported data, and compute the mAP, as in the official [scikit learn implementation](https://github.com/scikit-learn/scikit-learn/blob/baf0ea25d/sklearn/metrics/_ranking.py#L111)

(m)AP summarizes a precision-recall curve as the weighted mean of precisions achieved at each threshold, with the increase in recall from the previous threshold used as the weight:

$${AP} = \sum_n (R_n - R_{n-1}) P_n$$
    
where `P_n` and `R_n` are the precision and recall at the nth threshold. Using decreasing threshold values, the Recall $R_{n-1}$ at the first threshold is set to 0 as when the threshold is maximal, no detections are returned. Therefore, with no positives returned, the precision $P_{n-1}$ is by definition equal to 1.

*This implementation is not interpolated and is different from computing the area under the precision-recall curve with the trapezoidal rule, which uses linear interpolation and can be too optimistic.*

**Note:** this implementation is restricted to the binary classification task or multilabel classification task.

In [4]:
def clean_dataset_name(file_name,verbose=False):
    """
    return the name of the dataset wihtout the split extension
    """
    base_name = os.path.basename(file_name)
    dataset_name = base_name.split("_")[0][:-1]
    if verbose:
        print(dataset_name)
    return dataset_name

print("Found {} evaluation files.".format(len(input_files)))
    
all_nets_all_APs = []

prev_dataset = clean_dataset_name(input_files[0])
current_AP_group = []
training_datasets = [prev_dataset]

for use_state in range(len(input_files)):
    with open(input_files[use_state], 'rb') as f:
        data = pickle.load(f)

    all_training_states = []

    for elem in data:
        all_training_states.append(int(elem[0].split(".")[0].split("_")[-1]))

    all_training_states.sort()

    final_AP = []
    #print("\n",input_files[use_state])
    
    #all_nets_all_APs.append([input_files[use_state]])
    
    for dataset_idx in range(1,2):
        all_AP = []

        for model in data:
            print("\nProducing AP plot for {}\n".format(model[0]))
            print("With dataset {}\n".format(model[1][dataset_idx][0]))
            curve_coords = np.zeros([len(model[1:]),2])
            AP = 0
            R_n = 0
            for e, elem in reversed(list(enumerate(model[1:]))):
                curve_coords[e] = [elem[dataset_idx][6],elem[dataset_idx][5]]
                #print("thresh {}   Precision {}    Recall {}".format(elem[0],round(elem[dataset_idx][5],3),round(elem[dataset_idx][6],3)))
                AP += (elem[dataset_idx][6] - R_n) * elem[dataset_idx][5]
                R_n = elem[dataset_idx][6]

            #plt.plot(curve_coords[:,0],curve_coords[:,1])
            #print("AP: {}\n".format(AP))
            all_AP.append([model[1][dataset_idx][0],int(model[0].split(".")[0].split("_")[-1]),AP])

        #plt.show()

        all_AP.sort()
        #print(all_AP[-1])
        final_AP.append(all_AP[-1][-1])

    current_dataset = clean_dataset_name(input_files[use_state])
    if use_state == len(input_files)-1:
        current_AP_group.append(final_AP)
        all_nets_all_APs.append(current_AP_group)
        
    elif current_dataset != prev_dataset:
        prev_dataset = current_dataset
        all_nets_all_APs.append(current_AP_group)
        current_AP_group = []
        training_datasets.append(current_dataset)
        
    else:
        current_AP_group.append(final_AP)
    
        
"""
For the output shape we will produce the mean and standard deviation for each AP value for every model / dataset combination

        |                                 dataset
        |   base   base  bright bright  close  close  dark   drak  noisy  noisy
model   |   mean   std   mean   std     mean   std    mean   std   mean   std
____________________________________________________________________________________
bs1000  |
bs100   |
bs10    |
rb      |
ra      |
...     |

"""
output_AP = np.zeros([len(training_datasets),2])

dat = 0
for dataset_name, APs in zip(training_datasets,all_nets_all_APs):
    
    base_AP_mean = np.mean(np.array(APs)[:,0])   
    base_AP_std = np.std(np.array(APs)[:,0])
    
    output_AP[dat] = [base_AP_mean, base_AP_std]
    
    dat += 1
    # COMBINE WITH DATASET NAMES FOR QUICK OVERVIEW
    # NOW THROW IT ALL INTO ONE PANDAS DATAFRAME

Found 9 evaluation files.

Producing AP plot for OUTPUTS_TERMITES/tr1/tr1_yolov4_array_HPC_new_20000.pkl

With dataset real_termites_frame


Producing AP plot for OUTPUTS_TERMITES/tr2/tr2_yolov4_array_HPC_new_20000.pkl

With dataset real_termites_frame


Producing AP plot for OUTPUTS_TERMITES/tr3/tr3_yolov4_array_HPC_new_20000.pkl

With dataset real_termites_frame


Producing AP plot for OUTPUTS_TERMITES/tr4/tr4_yolov4_array_HPC_new_20000.pkl

With dataset real_termites_frame


Producing AP plot for OUTPUTS_TERMITES/tr5/tr5_yolov4_array_HPC_new_20000.pkl

With dataset real_termites_frame


Producing AP plot for OUTPUTS_TERMITES/ts1/ts1_yolov4_array_HPC_new_20000.pkl

With dataset real_termites_frame


Producing AP plot for OUTPUTS_TERMITES/ts2/ts2_yolov4_array_HPC_new_20000.pkl

With dataset real_termites_frame


Producing AP plot for OUTPUTS_TERMITES/ts3/ts3_yolov4_array_HPC_new_20000.pkl

With dataset real_termites_frame


Producing AP plot for OUTPUTS_TERMITES/ts4/ts4_yolov4_array_H

In [5]:
categories = [["desert_rec","desert_rec"],
              ["mean","std"]]

categories_tuples = list(zip(*categories))
columns = pd.MultiIndex.from_tuples(categories_tuples, names=["dataset","score"])
    
final_dataframe = pd.DataFrame(output_AP, index = training_datasets, columns=columns)

In [6]:
final_dataframe

dataset desert_rec          
score         mean       std
tr        0.999073  0.000172
ts        0.956142  0.001092

In [7]:
custom_name = "results_termtites_5_percent"

final_dataframe.to_csv(os.path.join(os.path.dirname(input_path),custom_name) + ".csv")

# IF the function below fails, this is likely due to exceeding the number of columns supported by HDF5 files!
# Restrict the number of simulated animals to < 20 if the goal is to train a DLC network

final_dataframe.to_hdf(
    os.path.join(os.path.dirname(input_path), custom_name) + ".h5",
    "df_with_missing",
    format="table",
    mode="w")
    